# setting

In [1]:
type_weight = {0:1,
               1:6,
               2:3}
type_weight_multipliers = type_weight
CSV_VER = 1
VER = 1 #matrix version

In [2]:
import pandas as pd, numpy as np
import pickle, glob, gc
from collections import Counter
import itertools
# multiprocessing 
import psutil
N_CORES = psutil.cpu_count()     # Available CPU cores
print(f"N Cores : {N_CORES}")
from multiprocessing import Pool

N Cores : 8


In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# Validation

In [4]:
type_labels = {'clicks':0, 'carts':1, 'orders':2}

def load_test(files):    
    dfs = []
    for e, chunk_file in enumerate(glob.glob(files)):
        chunk = pd.read_parquet(chunk_file)
        chunk.ts = (chunk.ts/1000).astype('int32')
        chunk['type'] = chunk['type'].map(type_labels).astype('int8')
        dfs.append(chunk)
    return pd.concat(dfs).reset_index(drop=True) #.astype({"ts": "datetime64[ms]"})

valid = load_test('/content/drive/MyDrive/Colab Notebooks/kaggle/OTTO/dataset/otto-validation/test_parquet/*')
print('Valid data has shape',valid.shape)
valid.head()

Valid data has shape (7683577, 4)


,session,aid,ts,type
0,11278654,1116864,1661185221,0
1,11278654,897528,1661259716,0
2,11278654,897528,1661337850,0
3,11278654,303312,1661338001,0
4,11278654,303312,1661338083,1


In [5]:
%%time

DISK_PIECES = 4
# LOAD THREE CO-VISITATION MATRICES
def pqt_to_dict(df):
    return df.groupby('aid_x').aid_y.apply(list).to_dict()

top_20_clicks = pqt_to_dict( pd.read_parquet(f'/content/drive/MyDrive/Colab Notebooks/kaggle/OTTO/dataset/co-visitation matrix/val/top_20_clicks_v{VER}_0.pqt') )
for k in range(1, DISK_PIECES): 
    top_20_clicks.update( pqt_to_dict( pd.read_parquet(f'/content/drive/MyDrive/Colab Notebooks/kaggle/OTTO/dataset/co-visitation matrix/val/top_20_clicks_v{VER}_{k}.pqt') ) )


top_20_buys = pqt_to_dict( pd.read_parquet(f'/content/drive/MyDrive/Colab Notebooks/kaggle/OTTO/dataset/co-visitation matrix/val/top_15_carts_orders_v{VER}_0.pqt') )
for k in range(1, DISK_PIECES): 
    top_20_buys.update( pqt_to_dict( pd.read_parquet(f'/content/drive/MyDrive/Colab Notebooks/kaggle/OTTO/dataset/co-visitation matrix/val/top_15_carts_orders_v{VER}_{k}.pqt') ) )
    
top_20_buy2buy = pqt_to_dict( pd.read_parquet(f'/content/drive/MyDrive/Colab Notebooks/kaggle/OTTO/dataset/co-visitation matrix/val/top_15_buy2buy_v{VER}_0.pqt') )

# TOP CLICKS AND ORDERS IN TEST
top_clicks = valid.loc[valid['type']==0, 'aid'].value_counts().index.values[:20]
top_carts = valid.loc[valid['type']== 1,'aid'].value_counts().index.values[:20]
top_orders = valid.loc[valid['type']==2, 'aid'].value_counts().index.values[:20]

print('Here are size of our 3 co-visitation matrices:')
print( len( top_20_clicks ), len( top_20_buy2buy ), len( top_20_buys ) )

Here are size of our 3 co-visitation matrices:
1812132 1055146 1812132
CPU times: user 1min 29s, sys: 6.58 s, total: 1min 35s
Wall time: 1min 54s


In [6]:
def df_parallelize_run(func, t_split):
    
    num_cores = np.min([N_CORES, len(t_split)])
    pool = Pool(num_cores)
    df = pool.map(func, t_split)
    pool.close()
    pool.join()
    
    return df

In [7]:
%%time
PIECES = 5
valid_bysession_list = []
for PART in range(PIECES):
    with open(f'/content/drive/MyDrive/Colab Notebooks/kaggle/OTTO/dataset/otto-valid-test-list/valid_group_tolist_{PART}_1.pkl', 'rb') as f:
        valid_bysession_list.extend(pickle.load(f))
print(len(valid_bysession_list))

1801251
CPU times: user 8.9 s, sys: 841 ms, total: 9.74 s
Wall time: 12.3 s


In [8]:
#type_weight_multipliers = {'clicks': 1, 'carts': 6, 'orders': 3}
def suggest_clicks(df):
    
    session = df[0]
    aids = df[1]
    types = df[2]
    unique_aids = list(dict.fromkeys(aids[::-1] ))
    # RERANK CANDIDATES USING WEIGHTS
    if len(unique_aids)>=20:
        weights=np.logspace(0.1,1,len(aids),base=2, endpoint=True)-1
        aids_temp = Counter() 
        # RERANK BASED ON REPEAT ITEMS AND TYPE OF ITEMS
        for aid,w,t in zip(aids,weights,types): 
            aids_temp[aid] += w * type_weight_multipliers[t]
        sorted_aids = [k for k,v in aids_temp.most_common(20)]
        return session, sorted_aids
    # USE "CLICKS" CO-VISITATION MATRIX
    aids2 = list(itertools.chain(*[top_20_clicks[aid] for aid in unique_aids if aid in top_20_clicks]))
    # RERANK CANDIDATES
    top_aids2 = [aid2 for aid2, cnt in Counter(aids2).most_common(20) if aid2 not in unique_aids]    
    result = unique_aids + top_aids2[:20 - len(unique_aids)]
    
    # USE TOP20 TEST CLICKS
    return session, result + list(top_clicks)[:20-len(result)]

In [9]:
%%time
# Predict on all sessions in parallel
temp = df_parallelize_run(suggest_clicks, valid_bysession_list)
val_clicks = pd.Series([f[1]  for f in temp], index=[f[0] for f in temp])
clicks_val_df = val_clicks.add_suffix("_clicks")

CPU times: user 28.9 s, sys: 5.83 s, total: 34.8 s
Wall time: 39.4 s


In [10]:
def suggest_carts(df):
    # User history aids and types
    session = df[0]
    aids = df[1]
    types = df[2]
    
    # UNIQUE AIDS AND UNIQUE BUYS
    unique_aids = list(dict.fromkeys(aids[::-1]))
    unique_buys = list(dict.fromkeys( [f for i, f in enumerate(aids) if types[i] in [0, 1]][::-1] ))
    
    # Rerank candidates using weights
    if len(unique_aids) >= 20:
        weights=np.logspace(0.5,1,len(aids),base=2, endpoint=True)-1
        aids_temp = Counter() 
        
        # Rerank based on repeat items and types of items
        for aid,w,t in zip(aids,weights,types): 
            aids_temp[aid] += w * type_weight_multipliers[t]
        
        # Rerank candidates using"top_20_carts" co-visitation matrix
        aids2 = list(itertools.chain(*[top_20_buys[aid] for aid in unique_buys if aid in top_20_buys]))
        for aid in aids2: aids_temp[aid] += 0.1
        sorted_aids = [k for k,v in aids_temp.most_common(20)]
        return session,sorted_aids
    # Use "cart order" and "clicks" co-visitation matrices
    aids1 = list(itertools.chain(*[top_20_clicks[aid] for aid in unique_aids if aid in top_20_clicks]))
    aids2 = list(itertools.chain(*[top_20_buys[aid] for aid in unique_aids if aid in top_20_buys]))
    
    # RERANK CANDIDATES
    top_aids2 = [aid2 for aid2, cnt in Counter(aids1+aids2).most_common(20) if aid2 not in unique_aids] 
    result = unique_aids + top_aids2[:20 - len(unique_aids)]
    
    # USE TOP20 TEST ORDERS
    return session, result + list(top_carts)[:20-len(result)]

In [11]:
%%time
# Predict on all sessions in parallel
temp = df_parallelize_run(suggest_carts, valid_bysession_list)
val_carts = pd.Series([f[1]  for f in temp], index=[f[0] for f in temp])
carts_val_df = val_carts.add_suffix("_carts")

CPU times: user 28.7 s, sys: 7.04 s, total: 35.8 s
Wall time: 56 s


In [12]:
def suggest_orders(df):
    # USE USER HISTORY AIDS AND TYPES
    session = df[0]
    aids = df[1]
    types = df[2]

    unique_aids = list(dict.fromkeys(aids[::-1] ))
    unique_buys = list(dict.fromkeys( [f for i, f in enumerate(aids) if types[i] in [1, 2]][::-1] ))
        # RERANK CANDIDATES USING WEIGHTS
    if len(unique_aids)>=20:
        
        weights=np.logspace(0.5,1,len(aids),base=2, endpoint=True)-1
        aids_temp = Counter() 
        # RERANK BASED ON REPEAT ITEMS AND TYPE OF ITEMS
        for aid,w,t in zip(aids,weights,types): 
            aids_temp[aid] += w * type_weight_multipliers[t]
        # RERANK CANDIDATES USING "BUY2BUY" CO-VISITATION MATRIX
        aids3 = list(itertools.chain(*[top_20_buy2buy[aid] for aid in unique_buys if aid in top_20_buy2buy]))
        for aid in aids3: aids_temp[aid] += 0.1
        sorted_aids = [k for k,v in aids_temp.most_common(20)]
        return session, sorted_aids
            
    # USE "CART ORDER" CO-VISITATION MATRIX
    aids2 = list(itertools.chain(*[top_20_buys[aid] for aid in unique_aids if aid in top_20_buys]))
    # USE "BUY2BUY" CO-VISITATION MATRIX
    aids3 = list(itertools.chain(*[top_20_buy2buy[aid] for aid in unique_buys if aid in top_20_buy2buy]))
    # RERANK CANDIDATES
    top_aids2 = [aid2 for aid2, cnt in Counter(aids2 + aids3).most_common(20) if aid2 not in unique_aids] 
    result = unique_aids + top_aids2[:20 - len(unique_aids)]
    # USE TOP20 TEST ORDERS
    return session, result + list(top_orders)[:20-len(result)]

In [13]:
%%time
# Predict on all sessions in parallel
temp = df_parallelize_run(suggest_orders, valid_bysession_list)
val_orders = pd.Series([f[1]  for f in temp], index=[f[0] for f in temp])
orders_val_df = val_orders.add_suffix("_orders")

CPU times: user 34.2 s, sys: 6.83 s, total: 41 s
Wall time: 57.2 s


In [14]:
val_pred_df = pd.concat([clicks_val_df, orders_val_df, carts_val_df]).reset_index()
val_pred_df.columns = ["session_type", "labels"]
val_pred_df["labels"] = val_pred_df.labels.apply(lambda x: " ".join(map(str,x)))
val_pred_df.to_parquet(f'/content/drive/MyDrive/Colab Notebooks/kaggle/OTTO/oof/handcrafted{CSV_VER}.pqt', index=False)

In [15]:
weights = {'clicks': 0.10, 'carts': 0.30, 'orders': 0.60}

valid_labels = pd.read_parquet('/content/drive/MyDrive/Colab Notebooks/kaggle/OTTO/dataset/otto-validation/test_labels.parquet')


def hits(b):
    # b[0] : session id
    # b[1] : ground truth
    # b[2] : aids prediction 
    return b[0], len(set(b[1]).intersection(set(b[2]))), np.clip(len(b[1]), 0, 20)

def otto_metric_piece(values, typ, verbose=True):
    
    c1 = pd.DataFrame(values, columns=["labels"]).reset_index().rename({"index":"session"}, axis=1)
    a = valid_labels.loc[valid_labels['type']==typ].merge(c1, how='left', on=['session'])

    b=[[a0, a1, a2] for a0, a1, a2 in zip(a["session"], a["ground_truth"], a["labels"])]
    c = df_parallelize_run(hits, b)
    c = np.array(c)
    
    recall = c[:,1].sum() / c[:,2].sum()
    
    print('{} recall = {:.5f}'.format(typ ,recall))
    
    return recall

def otto_metric(clicks, carts, orders, verbose = True):
    
    score = 0
    score += weights["clicks"] * otto_metric_piece(clicks, "clicks", verbose = verbose)
    score += weights["carts"] * otto_metric_piece(carts, "carts", verbose = verbose)
    score += weights["orders"] * otto_metric_piece(orders, "orders", verbose = verbose)
    if verbose:
        print('=============')
        print('Overall Recall = {:.5f}'.format(score))
        print('=============')
    
    return score

In [16]:
%%time
_ = otto_metric(val_clicks, val_carts, val_orders)

clicks recall = 0.52558
carts recall = 0.40965
orders recall = 0.64924
Overall Recall = 0.56500
CPU times: user 43.8 s, sys: 9.45 s, total: 53.3 s
Wall time: 54.4 s


In [17]:
del temp
_ = gc.collect()

In [18]:
# FREE MEMORY
del valid_bysession_list, val_clicks, val_carts,val_orders
del clicks_val_df,orders_val_df,carts_val_df,val_pred_df
del top_20_clicks, top_20_buy2buy, top_20_buys, top_clicks, top_orders, valid
_ = gc.collect()

# Test

In [19]:
test = load_test('/content/drive/MyDrive/Colab Notebooks/kaggle/OTTO/dataset/original/test_parquet/*')
print('Test data has shape',test.shape)

Test data has shape (6928123, 4)


,session,aid,ts,type
0,12899779,59625,1661724000,0
1,12899780,1142000,1661724000,0
2,12899780,582732,1661724058,0
3,12899780,973453,1661724109,0
4,12899780,736515,1661724136,0


In [20]:
%%time

top_20_clicks = pqt_to_dict( pd.read_parquet(f'/content/drive/MyDrive/Colab Notebooks/kaggle/OTTO/dataset/co-visitation matrix/test/top_20_clicks_v{VER}_0.pqt') )
for k in range(1, DISK_PIECES): 
    top_20_clicks.update( pqt_to_dict( pd.read_parquet(f'/content/drive/MyDrive/Colab Notebooks/kaggle/OTTO/dataset/co-visitation matrix/test/top_20_clicks_v{VER}_{k}.pqt') ) )


top_20_buys = pqt_to_dict( pd.read_parquet(f'/content/drive/MyDrive/Colab Notebooks/kaggle/OTTO/dataset/co-visitation matrix/test/top_15_carts_orders_v{VER}_0.pqt') )
for k in range(1, DISK_PIECES): 
    top_20_buys.update( pqt_to_dict( pd.read_parquet(f'/content/drive/MyDrive/Colab Notebooks/kaggle/OTTO/dataset/co-visitation matrix/test/top_15_carts_orders_v{VER}_{k}.pqt') ) )
    
top_20_buy2buy = pqt_to_dict( pd.read_parquet(f'/content/drive/MyDrive/Colab Notebooks/kaggle/OTTO/dataset/co-visitation matrix/test/top_15_buy2buy_v{VER}_0.pqt') )

# TOP CLICKS AND ORDERS IN TEST
top_clicks = test.loc[test['type']==0, 'aid'].value_counts().index.values[:20]
top_carts = test.loc[test['type']== 1,'aid'].value_counts().index.values[:20]
top_orders = test.loc[test['type']==2, 'aid'].value_counts().index.values[:20]

print('Here are size of our 3 co-visitation matrices:')
print( len( top_20_clicks ), len( top_20_buy2buy ), len( top_20_buys ) )

Here are size of our 3 co-visitation matrices:
1837166 1168768 1837166
CPU times: user 1min 27s, sys: 6.61 s, total: 1min 34s
Wall time: 1min 52s


In [21]:
%%time
PIECES = 5
test_bysession_list = []
for PART in range(PIECES):
    with open(f'/content/drive/MyDrive/Colab Notebooks/kaggle/OTTO/dataset/otto-valid-test-list/test_group_tolist_{PART}_1.pkl', 'rb') as f:
        test_bysession_list.extend(pickle.load(f))
print(len(test_bysession_list))

1671803
CPU times: user 12.1 s, sys: 773 ms, total: 12.9 s
Wall time: 16 s


In [22]:
%%time
# Predict on all sessions in parallel
temp = df_parallelize_run(suggest_clicks, test_bysession_list)
clicks_pred_df = pd.Series([f[1] for f in temp], index=[f[0] for f in temp])
clicks_pred_df = clicks_pred_df.add_suffix("_clicks")

CPU times: user 32.3 s, sys: 4.43 s, total: 36.7 s
Wall time: 41.9 s


In [23]:
%%time
# Predict on all sessions in parallel
temp = df_parallelize_run(suggest_carts, test_bysession_list)
carts_pred_df = pd.Series([f[1] for f in temp], index=[f[0] for f in temp])
carts_pred_df = carts_pred_df.add_suffix("_carts")

CPU times: user 29.1 s, sys: 4.72 s, total: 33.9 s
Wall time: 44 s


In [24]:
%%time
# Predict on all sessions in parallel
temp = df_parallelize_run(suggest_orders, test_bysession_list)
orders_pred_df = pd.Series([f[1] for f in temp], index=[f[0] for f in temp])
orders_pred_df = orders_pred_df.add_suffix("_orders")

CPU times: user 34.5 s, sys: 6.08 s, total: 40.6 s
Wall time: 44.5 s


In [25]:
pred_df = pd.concat([clicks_pred_df, orders_pred_df, carts_pred_df]).reset_index()
pred_df.columns = ["session_type", "labels"]
pred_df["labels"] = pred_df.labels.apply(lambda x: " ".join(map(str,x)))
pred_df.to_csv(f"handcrafted{CSV_VER}.csv", index=False)
pred_df.to_parquet(f'/content/drive/MyDrive/Colab Notebooks/kaggle/OTTO/submission/handcrafted{CSV_VER}.pqt', index=False)

In [ ]:
!pip install kaggle -q
import os
import json
f = open("/content/drive/MyDrive/Colab Notebooks/kaggle/kaggle.json", 'r')
json_data = json.load(f)
os.environ['KAGGLE_USERNAME'] = json_data['username']
os.environ['KAGGLE_KEY'] = json_data['key']

In [ ]:
!kaggle competitions submit -c otto-recommender-system -f handcrafted4.csv -m ""